In [1]:
#-*- encoding: utf-8 -*-

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import os.path
import sys
import numpy as np
import tensorflow as tf

import input_data
import models

/root/miniconda3/envs/tf1.13/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/root/miniconda3/envs/tf1.13/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/root/miniconda3/envs/tf1.13/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/root/miniconda3/envs/tf1.13/lib/python3.7/site-packages/tensorf

In [2]:
def data_stats(train_data, val_data, test_data):
    """mean and std_dev

        Args:
            train_data: (36923, 490)
            val_data: (4445, 490)
            test_data: (4890, 490)

        Return: (mean, std_dev)

        Result:
            mean: -3.975149608704592, 220.81257374779565
            std_dev: 0.8934739293234528
    """
    print(train_data.shape, val_data.shape, test_data.shape)
    all_data = np.concatenate((train_data, val_data, test_data), axis=0)
    std_dev = 255. / (all_data.max() - all_data.min())
    # mean_ = all_data.mean()
    mean_ = 255. * all_data.min() / (all_data.min() - all_data.max())
    return (mean_, std_dev)

def fp32_to_uint8(r):
    # method 1
    # s = (r.max() - r.min()) / 255.
    # z = 255. - r.max() / s
    # q = r / s + z

    # method 2
    std_dev = 0.8934739293234528
    mean_ = 220.81257374779565
    q = r / std_dev + mean_
    q = q.astype(np.uint8)
    return q

def calc(interpreter, input_data, label):

    # Get input and output tensors.
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    # print(input_details)
    # print(output_details)

    # Test model on random input data.
    # input_shape = input_details[0]['shape']
    # input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
    interpreter.set_tensor(input_details[0]['index'], input_data)

    interpreter.invoke()

    # The function `get_tensor()` returns a copy of the tensor data.
    # Use `tensor()` in order to get a pointer to the tensor.
    output_data = interpreter.get_tensor(output_details[0]['index'])
    # print(output_data)
    # print(label)

    return output_data

In [3]:
wanted_words = 'yes,no,up,down,left,right,on,off,stop,go'
sample_rate = 16000
clip_duration_ms = 1000
model_architecture = 'mobilenet-v3'
dct_coefficient_count = 10
batch_size = 1
window_size_ms = 40
window_stride_ms = 20
model_size_info = [4, 16, 10, 4, 2, 2, 16, 3, 3, 1, 1, 2, 32, 3, 3, 1, 1, 2, 32, 5, 5, 1, 1, 2]
data_url = 'http://download.tensorflow.org/data/speech_commands_v0.02.tar.gz'
data_dir = '/tmp/speech_dataset/'
silence_percentage = 10.0
unknown_percentage = 10.0
testing_percentage = 10
validation_percentage = 10

In [4]:
tf.logging.set_verbosity(tf.logging.INFO)
sess = tf.InteractiveSession()
words_list = input_data.prepare_words_list(wanted_words.split(','))
model_settings = models.prepare_model_settings(
    len(words_list), sample_rate, clip_duration_ms, window_size_ms,
    window_stride_ms, dct_coefficient_count)

audio_processor = input_data.AudioProcessor(
    data_url, data_dir, silence_percentage,
    unknown_percentage,
    wanted_words.split(','), validation_percentage,
    testing_percentage, model_settings)

label_count = model_settings['label_count']
fingerprint_size = model_settings['fingerprint_size']

fingerprint_input = tf.placeholder(
    tf.float32, [None, fingerprint_size], name='fingerprint_input')

logits = models.create_model(
    fingerprint_input,
    model_settings,
    model_architecture,
    model_size_info,
    is_training=False)

ground_truth_input = tf.placeholder(
    tf.float32, [None, label_count], name='groundtruth_input')

predicted_indices = tf.argmax(logits, 1)
expected_indices = tf.argmax(ground_truth_input, 1)
correct_prediction = tf.equal(predicted_indices, expected_indices)
confusion_matrix = tf.confusion_matrix(
    expected_indices, predicted_indices, num_classes=label_count)
evaluation_step = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))



Total number of layers: %d
Instructions for updating:
Use keras.layers.conv2d instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.average_pooling2d instead.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.


In [1471]:
from decimal import Decimal

class Transformer_bd(object):
    def __init__(self, precision):
        self.pre = precision
        self.shift_num = None

    def bTod(self, n):
        '''
        把一个带小数的二进制数n转换成十进制
        小数点后面保留pre位小数
        '''
        string_number1 = str(n) #number1 表示二进制数，number2表示十进制数
        decimal = 0  #小数部分化成二进制后的值
        flag = False   
        for i in string_number1: #判断是否含小数部分
            if i == '.':
                flag = True
                break
        if flag: #若二进制数含有小数部分
            string_integer, string_decimal = string_number1.split('.') #分离整数部分和小数部分
            for i in range(len(string_decimal)):
                decimal += 2**(-i-1)*int(string_decimal[i])  #小数部分化成二进制
            number2 = int(str(int(string_integer, 2))) + decimal
            return round(number2, self.pre)
        else: #若二进制数只有整数部分
            return int(string_number1, 2)#若只有整数部分 直接一行代码二进制转十进制 python还是骚 
    
    def dTob(self, n, shift=False):
        '''
        把十进制的浮点数n转换成二进制
        小数点后面保留pre位小数
        '''
        string_number1 = str(n) #number1 表示十进制数，number2表示二进制数
        flag = False   
        for i in string_number1: #判断是否含小数部分
            if i == '.':
                flag = True
                break
        if flag:
            string_integer, string_decimal = string_number1.split('.') #分离整数部分和小数部分
            integer = int(string_integer)
            decimal = Decimal(str(n)) - integer
            l1 = [0,1]
            l2 = []
            decimal_convert = ""
            while True:  
                if integer == 0: break
                x,y = divmod(integer, 2)  #x为商，y为余数 
                l2.append(y)
                integer = x
            string_integer = ''.join([str(j) for j in l2[::-1]])  #整数部分转换成二进制 
            i = 0  
            while decimal != 0 and i < self.pre:  
                result = int(decimal * 2)  
                decimal = decimal * 2 - result  
                decimal_convert = decimal_convert + str(result)  
                i = i + 1  
            string_number2 = string_integer + '.' + decimal_convert
            # return float(string_number2)

            # 这里进行移位修改
            if string_integer == '':
                string_integer = '0'

            if shift == True:
                string_number3 = decimal_convert.lstrip('0')
                lshift_num = len(decimal_convert) - len(string_number3) # 左移的位数
                self.shift_num = lshift_num
                return string_integer + '.' + string_number3
            else:
                return string_integer + '.' + decimal_convert

        else: #若十进制只有整数部分
            l1 = [0,1]
            l2 = []
            while True:  
                if n == 0: break
                x,y = divmod(n, 2)  #x为商，y为余数 
                l2.append(y)
                n = x
            string_number = ''.join([str(j) for j in l2[::-1]])  
            # return int(string_number)
            return string_number
        

    def right_shift(self, s):
        '''右移二进制字符串'''
        point_pos = s.find('.') # point_pos的值就说明小数点前面有多少个数字
        ss = s[:point_pos] + s[point_pos + 1:]
        if self.shift_num > point_pos:
            zero_fill = self.shift_num - point_pos   # 需要补0的个数
            for _ in range(zero_fill):
                ss = '0' + ss
            return '0.' + ss
        else:
            insert_pos = point_pos - self.shift_num
            ssl = list(ss)
            ssl.insert(insert_pos, '.')
            ss = ''.join(ssl)
            if insert_pos == 0:
                ss = '0' + ss   # 在头部补零
            return ss

    def lshift_decimal(self, num):
        '''得到左移后的十进制'''
        num = self.dTob(num, shift=True)
        num = self.bTod(num)
        return num

    def rshift_decimal(self, num):
        '''得到右移后的十进制'''
        num = self.dTob(num, shift=False)
        num = self.right_shift(num)
        num = self.bTod(num)
        return num

In [1473]:
a = 150
num = 0.001
print('a * b =', a*num)

b = Transformer_bd(32)
num2 = b.lshift_decimal(num)
print(num2)
print('before right shift, a * num2 =', a*num2)
print()

aft_shift = b.rshift_decimal(num2)
print(aft_shift)
num_aft = a * aft_shift
print('after right shift, a * b =', num_aft)

a * b = 0.15
0.5119999647140503
before right shift, a * num2 = 76.79999470710754

0.0009999999310821295
after right shift, a * b = 0.14999998966231942


In [1583]:
tflite_path = 'test_log/mobilenetv3_quant_eval/layers_lite_model/stem_conv.lite'
# tflite_path = 'test_log/mobilenetv3_quant_eval/layers_lite_model/inverted_residual_1_expansion.lite'
# tflite_path = 'test_log/mobilenetv3_quant_eval/layers_lite_model/inverted_residual_1_depthwise.lite'
# tflite_path = 'test_log/mobilenetv3_quant_eval/layers_lite_model/inverted_residual_1_projection.lite'
# tflite_path = 'test_log/mobilenetv3_quant_eval/layers_lite_model/inverted_residual_1_add.lite'
# tflite_path = 'test_log/mobilenetv3_quant_eval/layers_lite_model/inverted_residual_2_expansion.lite'
# tflite_path = 'test_log/mobilenetv3_quant_eval/layers_lite_model/inverted_residual_2_depthwise.lite'
# tflite_path = 'test_log/mobilenetv3_quant_eval/layers_lite_model/inverted_residual_2_projection.lite'
# tflite_path = 'test_log/mobilenetv3_quant_eval/layers_lite_model/inverted_residual_3_expansion.lite'
# tflite_path = 'test_log/mobilenetv3_quant_eval/layers_lite_model/inverted_residual_3_depthwise.lite'
# tflite_path = 'test_log/mobilenetv3_quant_eval/layers_lite_model/inverted_residual_3_projection.lite'
# tflite_path = 'test_log/mobilenetv3_quant_eval/layers_lite_model/inverted_residual_3_add.lite'
# tflite_path = 'test_log/mobilenetv3_quant_eval/layers_lite_model/AvgPool.lite'
# tflite_path = 'test_log/mobilenetv3_quant_eval/layers_lite_model/Conv2D.lite'

# Boss
# tflite_path = 'test_log/mobilenetv3_quant_eval/uint8input_8bit_calc_mean220_std0.89.lite'

# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path=tflite_path)
# interpreter = tf.lite.Interpreter(model_path="tflite_factory/swiftnet-uint8.lite")
interpreter.allocate_tensors()


In [1588]:
def manual_int(input_data):
    ################## stem conv ##################
    print('stem conv')
    new_data = input_data.reshape(-1, 49, 10, 1)
    new_data = tf.convert_to_tensor(new_data, tf.float32, name='input')
    new_data =  new_data - 221.
    s_iwr = tf.constant(0.0011018913937732577 / 0.16148914396762848, tf.float32)
    s_iwr = tf.cast(s_iwr, tf.float32)

    weight = np.load('test_log/mobilenetv3_quant_eval/weight/MBNetV3-CNN_stem_conv_conv_weights_quant_FakeQuantWithMinMaxVars.npy')
    bias = np.load('test_log/mobilenetv3_quant_eval/weight/MBNetV3-CNN_stem_conv_conv_Conv2D_Fold_bias.npy')
    print(weight.dtype, weight.shape)
    print(bias.dtype, bias.shape)

    weight = tf.convert_to_tensor(weight, tf.float32, name='weight')
    weight = weight - 132.
    weight = tf.transpose(weight, perm=[1,2,0,3])

    bias = tf.convert_to_tensor(bias, tf.float32, name='bias')
    print(weight)
    print(bias)

    output = tf.nn.depthwise_conv2d(new_data,  # 张量输入
                filter=weight, # 卷积核参数
                strides=[1,2,2,1], # 步长参数
                padding="SAME", # 卷积方式
                data_format=None,  # 数据格式，与步长参数配合，决定移动方式
                name='stem_conv') # 名字，用于tensorboard图形显示时使用

    output = tf.add(output, bias, name='add')
    output *= s_iwr
    output += 0.0035
    
    output = tf.nn.relu(output)
    output_uint8 = tf.math.round(output, name='round')
    output_uint8 = tf.cast(output_uint8, tf.uint8, name='uint8')
    add_2 = tf.identity(output_uint8)   # 给之后的做加法
    print()

    # ################## inverted residual 1 expansion ##################
    # print('inverted residual 1 expansion')
    # new_data = tf.cast(output_uint8, tf.float32)
    # s_iwr = tf.constant(0.003049441846087575 / 0.27361148595809937, tf.float16)
    # s_iwr = tf.cast(s_iwr, tf.float32)

    # weight = np.load('test_log/mobilenetv3_quant_eval/weight/MBNetV3-CNN_inverted_residual_1_expansion_conv_weights_quant_FakeQuantWithMinMaxVars.npy')
    # bias = np.load('test_log/mobilenetv3_quant_eval/weight/MBNetV3-CNN_inverted_residual_1_expansion_conv_Conv2D_Fold_bias.npy')
    # print(weight.dtype, weight.shape)
    # print(bias.dtype, bias.shape)

    # weight = tf.convert_to_tensor(weight, tf.float32)
    # weight -= 146
    # weight = tf.transpose(weight, perm=[1,2,3,0])
    # print(weight)

    # bias = tf.convert_to_tensor(bias, tf.float32)
    # print(bias)

    # output = tf.nn.conv2d(new_data,  # 张量输入
    #             filter=weight, # 卷积核参数
    #             strides=[1,1,1,1], # 步长参数
    #             padding="SAME", # 卷积方式
    #             data_format=None)  # 数据格式，与步长参数配合，决定移动方式
    # output = output + bias
    # # output += 0.0074
    # output *= s_iwr
    
    # output = tf.nn.relu(output)
    # output_uint8 = tf.math.round(output)
    # output_uint8 = tf.cast(output_uint8, tf.uint8)
    # print()

    # ################## inverted residual 1 depthwise ##################
    # print('inverted residual 1 depthwise')
    # new_data = tf.cast(output_uint8, tf.float32)
    # s_iwr = tf.constant(0.0045695919543504715 / 0.12676289677619934, tf.float16)
    # s_iwr = tf.cast(s_iwr, tf.float32)

    # weight = np.load('test_log/mobilenetv3_quant_eval/weight/MBNetV3-CNN_inverted_residual_1_depthwise_weights_quant_FakeQuantWithMinMaxVars.npy')
    # bias = np.load('test_log/mobilenetv3_quant_eval/weight/MBNetV3-CNN_inverted_residual_1_depthwise_depthwise_conv_Fold_bias.npy')
    # print(weight.dtype, weight.shape)
    # print(bias.dtype, bias.shape)

    # weight = tf.convert_to_tensor(weight, tf.float32)
    # weight -= 127
    # weight = tf.transpose(weight, perm=[1,2,3,0])
    # print(weight)

    # bias = tf.convert_to_tensor(bias, tf.float32)
    # print(bias)

    # output = tf.nn.depthwise_conv2d(new_data,  # 张量输入
    #             filter=weight, # 卷积核参数
    #             strides=[1,1,1,1], # 步长参数
    #             padding="SAME", # 卷积方式
    #             data_format=None)  # 数据格式，与步长参数配合，决定移动方式
    # output = output + bias
    # # output += 0.0301
    # output *= s_iwr
    
    # output = tf.nn.relu(output)
    # output_uint8 = tf.math.round(output)
    # output_uint8 = tf.cast(output_uint8, tf.uint8)
    # print()

    # ################## inverted residual 1 projection ##################
    # print('inverted residual 1 projection')
    # new_data = tf.cast(output_uint8, tf.float32)
    # s_iwr = tf.constant(0.0009397256653755903 / 0.16901935636997223, tf.float16)
    # s_iwr = tf.cast(s_iwr, tf.float32)

    # weight = np.load('test_log/mobilenetv3_quant_eval/weight/MBNetV3-CNN_inverted_residual_1_projection_conv_weights_quant_FakeQuantWithMinMaxVars.npy')
    # bias = np.load('test_log/mobilenetv3_quant_eval/weight/MBNetV3-CNN_inverted_residual_1_projection_conv_Conv2D_Fold_bias.npy')
    # print(weight.dtype, weight.shape)
    # print(bias.dtype, bias.shape)

    # weight = tf.convert_to_tensor(weight, tf.float32)
    # weight -= 101
    # weight = tf.transpose(weight, perm=[1,2,3,0])
    # print(weight)

    # bias = tf.convert_to_tensor(bias, tf.float32)
    # print(bias)

    # output = tf.nn.conv2d(new_data,  # 张量输入
    #             filter=weight, # 卷积核参数
    #             strides=[1,1,1,1], # 步长参数
    #             padding="SAME", # 卷积方式
    #             data_format=None)  # 数据格式，与步长参数配合，决定移动方式
    # output = output + bias
    # # output += 0.00052
    # output = output * s_iwr + 133
    
    # output_uint8 = tf.math.round(output)
    # output_uint8 = tf.cast(output_uint8, tf.uint8)
    # add_1 = tf.identity(output_uint8)
    # print()

    # ################## inverted residual 1 add ##################
    # add_1 = tf.cast(add_1, tf.float32)
    # add_2 = tf.cast(add_2, tf.float32)

    # add_1 = tf.constant(0.16901935636997223, tf.float32) * (add_1 - 133)
    # add_2 = tf.constant(0.16148914396762848, tf.float32) * add_2

    # output_result = tf.add(add_1, add_2)
    # output = output_result / tf.constant(0.24699252843856812, tf.float32) + 89
    # output_uint8 = tf.math.round(output)
    # output_uint8 = tf.cast(output_uint8, tf.uint8)

    # ################## inverted residual 2 expansion ##################
    # print('inverted residual 2 expansion')
    # new_data = tf.cast(output_uint8, tf.float32)
    # new_data -= 89
    # s_iwr = tf.constant(0.0020657109562307596 / 0.09814818948507309, tf.float16)
    # s_iwr = tf.cast(s_iwr, tf.float32)

    # weight = np.load('test_log/mobilenetv3_quant_eval/weight/MBNetV3-CNN_inverted_residual_2_expansion_conv_weights_quant_FakeQuantWithMinMaxVars.npy')
    # bias = np.load('test_log/mobilenetv3_quant_eval/weight/MBNetV3-CNN_inverted_residual_2_expansion_conv_Conv2D_Fold_bias.npy')
    # print(weight.dtype, weight.shape)
    # print(bias.dtype, bias.shape)

    # weight = tf.convert_to_tensor(weight, tf.float32)
    # weight -= 149
    # weight = tf.transpose(weight, perm=[1,2,3,0])
    # print(weight)

    # bias = tf.convert_to_tensor(bias, tf.float32)
    # print(bias)

    # output = tf.nn.conv2d(new_data,  # 张量输入
    #             filter=weight, # 卷积核参数
    #             strides=[1,1,1,1], # 步长参数
    #             padding="SAME", # 卷积方式
    #             data_format=None)  # 数据格式，与步长参数配合，决定移动方式
    # output = output + bias
    # # output += 0.01062
    # output *= s_iwr

    # output = tf.nn.relu(output)
    # output_uint8 = tf.math.round(output)
    # output_uint8 = tf.cast(output_uint8, tf.uint8)
    # print()

    # ################## inverted residual 2 depthwise ##################
    # print('inverted residual 2 depthwise')
    # new_data = tf.cast(output_uint8, tf.float32)
    # s_iwr = tf.constant(0.0014443636173382401 / 0.062810979783535, tf.float16)
    # s_iwr = tf.cast(s_iwr, tf.float32)

    # weight = np.load('test_log/mobilenetv3_quant_eval/weight/MBNetV3-CNN_inverted_residual_2_depthwise_weights_quant_FakeQuantWithMinMaxVars.npy')
    # bias = np.load('test_log/mobilenetv3_quant_eval/weight/MBNetV3-CNN_inverted_residual_2_depthwise_depthwise_conv_Fold_bias.npy')
    # print(weight.dtype, weight.shape)
    # print(bias.dtype, bias.shape)

    # weight = tf.convert_to_tensor(weight, tf.float32)
    # weight -= 120
    # weight = tf.transpose(weight, perm=[1,2,3,0])
    # print(weight)

    # bias = tf.convert_to_tensor(bias, tf.float32)
    # print(bias)

    # output = tf.nn.depthwise_conv2d(new_data,  # 张量输入
    #             filter=weight, # 卷积核参数
    #             strides=[1,1,1,1], # 步长参数
    #             padding="SAME", # 卷积方式
    #             data_format=None)  # 数据格式，与步长参数配合，决定移动方式
    # output = output + bias
    # # output += 0.0153
    # output *= s_iwr
    
    # output = tf.nn.relu(output)
    # output_uint8 = tf.math.round(output)
    # output_uint8 = tf.cast(output_uint8, tf.uint8)
    # print()

    # ################## inverted residual 2 projection ##################
    # print('inverted residual 2 projection')
    # new_data = tf.cast(output_uint8, tf.float32)
    # s_iwr = tf.constant(0.00040918667218647897 / 0.0929793044924736, tf.float16)
    # s_iwr = tf.cast(s_iwr, tf.float32)

    # weight = np.load('test_log/mobilenetv3_quant_eval/weight/MBNetV3-CNN_inverted_residual_2_projection_conv_weights_quant_FakeQuantWithMinMaxVars.npy')
    # bias = np.load('test_log/mobilenetv3_quant_eval/weight/MBNetV3-CNN_inverted_residual_2_projection_conv_Conv2D_Fold_bias.npy')
    # print(weight.dtype, weight.shape)
    # print(bias.dtype, bias.shape)

    # weight = tf.convert_to_tensor(weight, tf.float32)
    # weight -= 148
    # weight = tf.transpose(weight, perm=[1,2,3,0])
    # print(weight)

    # bias = tf.convert_to_tensor(bias, tf.float32)
    # print(bias)

    # output = tf.nn.conv2d(new_data,  # 张量输入
    #             filter=weight, # 卷积核参数
    #             strides=[1,1,1,1], # 步长参数
    #             padding="SAME", # 卷积方式
    #             data_format=None)  # 数据格式，与步长参数配合，决定移动方式
    # output = output + bias
    # output = output * s_iwr + 138
    
    # output_uint8 = tf.math.round(output)
    # output_uint8 = tf.cast(output_uint8, tf.uint8)
    # add_2 = tf.identity(output_uint8)
    # print()

    # ################## inverted residual 3 expansion ##################
    # print('inverted residual 3 expansion')
    # new_data = tf.cast(output_uint8, tf.float32)
    # new_data -= 138
    # s_iwr = tf.constant(0.0005567758926190436 / 0.07842949777841568, tf.float16)
    # s_iwr = tf.cast(s_iwr, tf.float32)

    # weight = np.load('test_log/mobilenetv3_quant_eval/weight/MBNetV3-CNN_inverted_residual_3_expansion_conv_weights_quant_FakeQuantWithMinMaxVars.npy')
    # bias = np.load('test_log/mobilenetv3_quant_eval/weight/MBNetV3-CNN_inverted_residual_3_expansion_conv_Conv2D_Fold_bias.npy')
    # print(weight.dtype, weight.shape)
    # print(bias.dtype, bias.shape)

    # weight = tf.convert_to_tensor(weight, tf.float32)
    # weight -= 137
    # weight = tf.transpose(weight, perm=[1,2,3,0])
    # print(weight)

    # bias = tf.convert_to_tensor(bias, tf.float32)
    # print(bias)

    # output = tf.nn.conv2d(new_data,  # 张量输入
    #             filter=weight, # 卷积核参数
    #             strides=[1,1,1,1], # 步长参数
    #             padding="SAME", # 卷积方式
    #             data_format=None)  # 数据格式，与步长参数配合，决定移动方式
    # output = output + bias
    # # output += 0.00113
    # output *= s_iwr
    
    # output = tf.nn.relu(output)
    # output_uint8 = tf.math.round(output)
    # output_uint8 = tf.cast(output_uint8, tf.uint8)
    # print()

    # ################## inverted residual 3 depthwise ##################
    # print('inverted residual 3 depthwise')
    # new_data = tf.cast(output_uint8, tf.float32)
    # s_iwr = tf.constant(0.013642110861837864 / 0.05131378769874573, tf.float16)
    # s_iwr = tf.cast(s_iwr, tf.float32)

    # weight = np.load('test_log/mobilenetv3_quant_eval/weight/MBNetV3-CNN_inverted_residual_3_depthwise_weights_quant_FakeQuantWithMinMaxVars.npy')
    # bias = np.load('test_log/mobilenetv3_quant_eval/weight/MBNetV3-CNN_inverted_residual_3_depthwise_depthwise_conv_Fold_bias.npy')
    # print(weight.dtype, weight.shape)
    # print(bias.dtype, bias.shape)

    # weight = tf.convert_to_tensor(weight, tf.float32)
    # weight -= 79
    # weight = tf.transpose(weight, perm=[1,2,3,0])
    # print(weight)

    # bias = tf.convert_to_tensor(bias, tf.float32)
    # print(bias)

    # output = tf.nn.depthwise_conv2d(new_data,  # 张量输入
    #             filter=weight, # 卷积核参数
    #             strides=[1,1,1,1], # 步长参数
    #             padding="SAME", # 卷积方式
    #             data_format=None)  # 数据格式，与步长参数配合，决定移动方式
    # output = output + bias
    # output *= s_iwr
    
    # output = tf.nn.relu(output)
    # output_uint8 = tf.math.round(output)
    # output_uint8 = tf.cast(output_uint8, tf.uint8)
    # print()

    # ################## inverted residual 3 projection ##################
    # print('inverted residual 3 projection')
    # new_data = tf.cast(output_uint8, tf.float32)
    # s_iwr = tf.constant(0.0008600406581535935 / 0.20826007425785065, tf.float16)
    # s_iwr = tf.cast(s_iwr, tf.float32)

    # weight = np.load('test_log/mobilenetv3_quant_eval/weight/MBNetV3-CNN_inverted_residual_3_projection_conv_weights_quant_FakeQuantWithMinMaxVars.npy')
    # bias = np.load('test_log/mobilenetv3_quant_eval/weight/MBNetV3-CNN_inverted_residual_3_projection_conv_Conv2D_Fold_bias.npy')
    # print(weight.dtype, weight.shape)
    # print(bias.dtype, bias.shape)

    # weight = tf.convert_to_tensor(weight, tf.float32)
    # weight -= 125
    # weight = tf.transpose(weight, perm=[1,2,3,0])
    # print(weight)

    # bias = tf.convert_to_tensor(bias, tf.float32)
    # print(bias)

    # output = tf.nn.conv2d(new_data,  # 张量输入
    #             filter=weight, # 卷积核参数
    #             strides=[1,1,1,1], # 步长参数
    #             padding="SAME", # 卷积方式
    #             data_format=None)  # 数据格式，与步长参数配合，决定移动方式
    # output = output + bias
    # output = output * s_iwr + 133
    
    # output_uint8 = tf.math.round(output)
    # output_uint8 = tf.cast(output_uint8, tf.uint8)
    # add_1 = tf.identity(output_uint8)
    # print()

    # ################## inverted residual 3 add ##################
    # add_1 = tf.cast(add_1, tf.float32)
    # add_2 = tf.cast(add_2, tf.float32)

    # add_1 = tf.constant(0.20826007425785065, tf.float32) * (add_1 - 133)
    # add_2 = tf.constant(0.0929793044924736, tf.float32) * (add_2 - 138)

    # output_result = tf.add(add_1, add_2)
    # output_uint8 = output_result / tf.constant(0.21021947264671326, tf.float32) + 131
    # output_uint8 = tf.math.round(output_uint8)
    # output_uint8 = tf.cast(output_uint8, tf.uint8)

    # ################## AvgPool ##################
    # # method 1
    # # new_data = tf.cast(output_uint8, tf.float32)
    # # new_data = 0.21021947264671326 * (new_data - 131)
    # # output = tf.nn.avg_pool(new_data,
    # #                 ksize=[1,25,5,1],
    # #                 strides=[1,25,5,1],
    # #                 padding='VALID')
    # # output = output / 0.21021947264671326 + 131
    # # output_uint8 = tf.math.round(output)
    # # output_uint8 = tf.cast(output, tf.uint8)

    # # method 2 (简化版本，发现scale和zero_point完全可以消除)
    # new_data = tf.cast(output_uint8, tf.float32)
    # output = tf.nn.avg_pool(new_data,
    #                 ksize=[1,25,5,1],
    #                 strides=[1,25,5,1],
    #                 padding='VALID')
    # # output -= 0.0041
    # output_uint8 = tf.math.round(output)
    # output_uint8 = tf.cast(output_uint8, tf.uint8)

    # ################## Conv2D ##################
    # print('Conv2D')
    # new_data = tf.cast(output_uint8, tf.float32)
    # new_data -= 131
    # s_iwr = tf.constant(0.0033858332317322493 / 0.1784215271472931, tf.float16)
    # s_iwr = tf.cast(s_iwr, tf.float32)

    # weight = np.load('test_log/mobilenetv3_quant_eval/weight/MBNetV3-CNN_fc_conv_weights_quant_FakeQuantWithMinMaxVars.npy')
    # bias = np.load('test_log/mobilenetv3_quant_eval/weight/MBNetV3-CNN_fc_conv_Conv2D_bias.npy')
    # print(weight.dtype, weight.shape)
    # print(bias.dtype, bias.shape)

    # weight = tf.convert_to_tensor(weight, tf.float32)
    # weight -= 143
    # weight = tf.transpose(weight, perm=[1,2,3,0])
    # print(weight)

    # bias = tf.convert_to_tensor(bias, tf.float32)
    # print(bias)

    # output = tf.nn.conv2d(new_data,  # 张量输入
    #             filter=weight, # 卷积核参数
    #             strides=[1,1,1,1], # 步长参数
    #             padding="SAME", # 卷积方式
    #             data_format=None)  # 数据格式，与步长参数配合，决定移动方式
    # output = output + bias
    # output = output * s_iwr + 129
    
    # output_uint8 = tf.math.round(output)
    # output_uint8 = tf.cast(output_uint8, tf.uint8)
    # add_1 = tf.identity(output_uint8)
    # print()

    # ################## Reshape ##################
    # output_uint8 = tf.squeeze(output_uint8, axis=[1,2])

    # ################## Softmax ##################
    # new_data = tf.cast(output_uint8, tf.float32)
    # new_data = tf.constant(0.1784215271472931, tf.float32) * (new_data - 129)
    # output = tf.nn.softmax(new_data)
    # output = output / tf.constant(0.00390625, tf.float32)

    # output_uint8 = tf.math.round(output)
    # output_uint8 = tf.cast(output_uint8, tf.uint8)

    ################## running ##################

    return output_uint8.eval(), output.eval()

In [1589]:
def debug(output_uint8):
    add_2 = np.load('test_log/mobilenetv3_quant_eval/debug/add_2.npy')
    ################## inverted residual 1 add ##################
    add_1 = tf.cast(output_uint8, tf.float32)
    add_2 = tf.convert_to_tensor(add_2, tf.float32)
    add_1_scale = tf.constant(0.16901935636997223, tf.float32)
    add_2_scale = tf.constant(0.16148914396762848, tf.float32)

    add_1 = add_1_scale * (add_1 - 133)
    add_2 = add_2_scale * add_2

    # low = tf.ones_like(add_1) * -22.47957420349121
    # high = tf.ones_like(add_1) * 20.620361328125
    # add_1 = tf.where(add_1 < -22.47957420349121, low, add_1)
    # add_1 = tf.where(add_1 > 20.620361328125, high, add_1)

    # high = tf.ones_like(add_2) * 20.620361328125
    # add_2 = tf.where(add_2 > 20.620361328125, high, add_2)

    output_result = tf.math.add(add_1, add_2)
    output_scale = tf.constant(0.24699252843856812, tf.float32)
    output = output_result / output_scale + 89
    output_uint8 = tf.math.round(output)
    # output_uint8 = tf.math.floor(output)
    output_uint8 = tf.cast(output_uint8, tf.uint8)

    return output_uint8.eval(), output.eval()

In [1590]:
def manual_real(input_data):
    '''
    直接用float32复现
    '''
    ################## stem conv ##################
    print('stem conv')
    precision = 32
    new_data = input_data.reshape(-1, 49, 10, 1)
    new_data = tf.convert_to_tensor(new_data, tf.float32, name='input')
    new_data =  new_data - 221.
    # s_iw = tf.constant(1.1192268133163452, tf.float32) * tf.constant(0.0009845112217590213, tf.float32)
    s_iwr = tf.constant(0.0011018913937732577 / 0.16148914396762848, tf.float16)
    s_iwr = tf.cast(s_iwr, tf.float32)

    weight = np.load('test_log/mobilenetv3_quant_eval/weight/MBNetV3-CNN_stem_conv_conv_weights_quant_FakeQuantWithMinMaxVars.npy')
    bias = np.load('test_log/mobilenetv3_quant_eval/weight/MBNetV3-CNN_stem_conv_conv_Conv2D_Fold_bias.npy')
    print(weight.dtype, weight.shape)
    print(bias.dtype, bias.shape)

    weight = tf.convert_to_tensor(weight, tf.float32, name='weight')
    weight = weight - 132.
    weight = tf.transpose(weight, perm=[1,2,0,3])

    bias = tf.convert_to_tensor(bias, tf.float32, name='bias')
    print(weight)
    print(bias)

    output = tf.nn.depthwise_conv2d(new_data,  # 张量输入
                filter=weight, # 卷积核参数
                strides=[1,2,2,1], # 步长参数
                padding="SAME", # 卷积方式
                data_format=None,  # 数据格式，与步长参数配合，决定移动方式
                name='stem_conv') # 名字，用于tensorboard图形显示时使用
    output_fp = output

    output_add = tf.add(output_fp, bias, name='add')
    output_add = output_add * s_iwr
    
    output = tf.nn.relu(output_add)
    output_uint8 = tf.math.round(output, name='round')
    # output_uint8 = tf.math.floor(output, name='round')
    output_uint8 = tf.cast(output_uint8, tf.uint8, name='uint8')
    add_2 = tf.identity(output_uint8)   # 给之后的做加法
    # np.save('test_log/mobilenetv3_quant_eval/debug/add_2.npy', add_2.eval())
    print()

    # output_uint8 = sess.run(output_uint8)
    return output_uint8.eval(), output.eval()

In [1591]:
# test set
set_size = audio_processor.set_size('testing')
tf.logging.info('set_size=%d', set_size)
total_accuracy = 0
for i in range(0, set_size, batch_size):
    test_fingerprints, test_ground_truth = audio_processor.get_data(
        batch_size, i, model_settings, 0.0, 0.0, 0, 'testing', sess)
    # print(test_fingerprints.shape)  # (batch_size 490)
    # print(test_ground_truth.shape)  # (batch_size, 12)
    test_fingerprints = fp32_to_uint8(test_fingerprints)
    output_simulate, output_ = manual_int(test_fingerprints)
    # output_simulate, output_ = manual_real(test_fingerprints)
    output_real = calc(interpreter, test_fingerprints, test_ground_truth)
    # output_simulate, output_ = debug(output_real)

    # real_output
    # np.save('test_log/mobilenetv3_quant_eval/debug/add_2.npy', output_real)
    # np.save('test_log/mobilenetv3_quant_eval/debug/output_real.npy', output_real)
    # output_real = np.load('test_log/mobilenetv3_quant_eval/debug/output_real.npy')

    # print(output_simulate.shape)
    # print(output_real.shape)
    # print(output_simulate)
    # print(output_real)
    # print(test_ground_truth)

    # sys.exit(0)
    neq = output_simulate != output_real
    print(output_[neq])
    print(output_real[neq])
    print('add', sorted(output_real[neq] - output_[neq]))
    # print('sub', sorted(output_[neq] - output_real[neq]))
    
    eq = tf.equal(output_real, output_simulate)
    mask = tf.cast(tf.zeros_like(eq), tf.bool)
    neq = tf.reduce_sum(tf.cast(tf.equal(eq, mask), tf.int32))
    print(sess.run(neq), '/', sess.run(tf.size(eq)))

    sys.exit(0)
    # batch_size = min(FLAGS.batch_size, set_size - i)


'''
############################### get all data mean and std_dev ###############################
training_fingerprints, training_ground_truth = audio_processor.get_data(
    -1, 0, model_settings, 0.0, 0.0, 0, 'training', sess)
validation_fingerprints, validation_ground_truth = audio_processor.get_data(
    -1, 0, model_settings, 0.0, 0.0, 0, 'validation', sess)
testing_fingerprints, testing_ground_truth = audio_processor.get_data(
    -1, 0, model_settings, 0.0, 0.0, 0, 'testing', sess)
mean_, std_dev = data_stats(training_fingerprints, validation_fingerprints, testing_fingerprints)
print(mean_, std_dev)
'''

INFO:tensorflow:set_size=4890
stem conv
uint8 (1, 10, 4, 16)
int32 (16,)
Tensor("transpose_1582:0", shape=(10, 4, 1, 16), dtype=float32)
Tensor("bias_557:0", shape=(16,), dtype=float32)

[ 0.49810204  5.4995923   4.496565    4.496565    7.498824    2.4973335
  5.4995923  11.497287    0.49810204  2.4973335  18.498009  ]
[ 1  6  5  5  8  3  6 12  1  3 19]
add [0.5004077, 0.5004077, 0.5011759, 0.50189793, 0.50189793, 0.5019913, 0.5026665, 0.5026665, 0.5027132, 0.50343513, 0.50343513]
11 / 2000


SystemExit: 0